In [1]:
## Func for confs

import os
from pathlib import Path
# doubletap
os.environ['OE_LICENSE']=os.environ.get('OE_LICENSE', os.environ['HOME2']+'/ASAP-oe_license.txt')
assert Path(os.environ['OE_LICENSE']).exists()
from openeye import oechem
from openeye import oeomega

os.environ['OE_LICENSE']

# copypasted boilerplate....
dense = True
omegaOpts = oeomega.OEOmegaOptions(oeomega.OEOmegaSampling_Dense) if dense else oeomega.OEOmegaOptions()
opts = oechem.OESimpleAppOptions(omegaOpts, "Omega", oechem.OEFileStringType_Mol, oechem.OEFileStringType_Mol3D)
#print(oechem.OEConfigureOpts(opts, [], False))
omegaOpts.UpdateValues(opts)
omega = oeomega.OEOmega(omegaOpts)

flipperOpts = oeomega.OEFlipperOptions()
opts = oechem.OESimpleAppOptions(flipperOpts, "stereo_and_torsion", oechem.OEFileStringType_Mol, oechem.OEFileStringType_Mol)
flipperOpts.UpdateValues(opts)

ret_codes = {getattr(oeomega, v) : v.lstrip('OEOmegaReturnCode_') for v in dir(oeomega) if 'OEOmegaReturnCode' in v}
ret_code_tallies = {c: 0 for c in ret_codes.values()}

def gen_conf(mol: oechem.OEMol) -> None:
    ret_code = omega.Build(mol)
    ret_code_tallies[ret_codes[ret_code]] += 1
    if ret_code == oeomega.OEOmegaReturnCode_UnspecifiedStereo:
        for enantiomer in oeomega.OEFlipper(mol.GetActive(), flipperOpts):
            gen_conf(mol)
    elif ret_code  == oeomega.OEOmegaReturnCode_Success:
        oechem.OEWriteMolecule(ofs, mol)
    else:
        pass

In [2]:
import os
from pathlib import Path

for path in Path('.').glob('*.sdf'):
    in_filename = path.as_posix()
    out_filename = f'{os.path.splitext(path)[0]}.oeb.gz'
    ifs = oechem.oemolistream()
    assert ifs.open(in_filename)
    ofs = oechem.oemolostream()
    assert ofs.open(out_filename)
    for mol in ifs.GetOEMols():
        mol.ClearCoords()
        gen_conf(mol)
    ofs.close()

In [13]:
from rdkit import Chem
from rdkit.Chem import AllChem
import contextlib

with Chem.SDMolSupplier('Enamine_Fragment_Collection_259380cmpds_20230413.sdf') as mol_sup,\
    Chem.SDWriter('Enamine_Fragment_Collection_halo-subset.sdf') as writer:
    mol: Chem.Mol
    for mol in mol_sup:    
        with contextlib.suppress(Exception):
            n = AllChem.CalcNumRotatableBonds( mol )
            if n > 3:
                continue
            if not mol.HasSubstructMatch(Chem.MolFromSmarts('[I,Br,Cl]')):
                continue
            writer.write(mol)

In [ ]:
in_filename = 'Carboxylic Acid Bioisosteres_1817cmpds.sdf'
out_filename = 'Enamine_carboxyl-bioisosteres.oeb.gz'
ifs = oechem.oemolistream()
assert ifs.open(in_filename)
ofs = oechem.oemolostream()
assert ofs.open(out_filename)
for mol in ifs.GetOEMols():
    mol.ClearCoords()
    gen_conf(mol)
ofs.close()

In [ ]:
in_filename = 'Sulfonamides_2839cmpds.sdf'
out_filename = 'Enamine_sulfonamides.oeb.gz'
ifs = oechem.oemolistream()
assert ifs.open(in_filename)
ofs = oechem.oemolostream()
assert ofs.open(out_filename)
for mol in ifs.GetOEMols():
    mol.ClearCoords()
    gen_conf(mol)
ofs.close()